In [1]:
!pwd

/home/peterr/macocu/task1/task2


In [1]:
import pandas as pd
import numpy as np
import torch



def read_file(fname: str, correct_labels=False) -> pd.DataFrame:
    """Reads a filename, return df with text and labels."""

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0
        df["labels"] = df.labels.astype(int)
    df = df.drop(columns=["role"])
    return df

en_test, en_train = "../data/merged-en.test.tsv" , "../data/merged-en.train.tsv"
hr_test, hr_train = "../data/merged-hr.test.tsv" , "../data/merged-hr.train.tsv"
sl_test, sl_train = "../data/merged-sl.test.tsv",  "../data/merged-sl.train.tsv"


In [5]:
def evaluate_HF_model(model_type, model_name, language):
    from simpletransformers.classification import ClassificationModel
    
    model = ClassificationModel(model_type, model_name)
    
    test_fname = f"../data/merged-{lang}.test.tsv"
    test = read_file(test_fname, correct_labels = True)
    
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]

    accuracy = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true, y_pred, average="macro" )
    
    return accuracy, f1


lang = "hr"
model_type = "electra"
model_name = "./finetuned_models/HR_hate___classla_bcms-bertic_6/"

accs = list()
f1s = list()
for i in range(1):
    import gc
    gc.collect()
    a, f = evaluate_HF_model(model_type, model_name, lang)
    accs.append(a)
    f1s.append(f)


Some weights of the model checkpoint at ./finetuned_models/HR_hate___classla_bcms-bertic_6/ were not used when initializing ElectraForSequenceClassification: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ./finetuned_models/HR_hate___classla_bcms-bertic_6/ and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'pooler.d

# Added pretraining:

In [5]:
def train_and_evaluate_HF_model(model_type, model_name, language):
    from simpletransformers.classification import ClassificationModel
    
    train_fname = f"../data/merged-{lang}.train.tsv"
    train = read_file(train_fname, correct_labels = True)
    
    
    model_args = {
        "num_train_epochs": 5,
        "learning_rate": 1e-5,
        "overwrite_output_dir": True,
        "train_batch_size": 40
    }

    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args

    )

    model.overwrite_output_dir = True
    #model.train_model(train, )
    
    test_fname = f"../data/merged-{lang}.test.tsv"
    test = read_file(test_fname, correct_labels = True)
    
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    #y_pred = model.predict(list(test["text"].values))[0]    
    accs = list()
    f1s = list()
    for i in range(10):
        model.train_model(train, )
        y_pred = model.predict(list(test["text"].values))[0]
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro" )
        accs.append(accuracy)
        f1s.append(f1)
    print(f"""
# Model: {model_name}

|language|accuracy|f1 score|
|---|---|---|""", end="")
    for a, f in zip(accs, f1s):
        print(f"|{lang}|{a:0.3}|{f:0.3}|")
    return model, accs, f1s


lang = "hr"
model_type = "electra"
model_name = "./finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5"

model, accs, f1s = train_and_evaluate_HF_model(model_type, model_name, lang)



model_name2 = "EMBEDDIA/crosloengual-bert"
model_type2 = "bert"
model2, accs2, f1s2 = train_and_evaluate_HF_model(model_type2, model_name2, lang)
print(model_name, "\nAccuracies: ", accs, "\nF1s: ", f1s)
print(model_name2, "\nAccuracies: ", accs2, "\nF1s: ", f1s2)

Some weights of the model checkpoint at ./finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5 were not used when initializing ElectraForSequenceClassification: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ./finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5 and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias



# Model: ./finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5

|language|accuracy|f1 score|
|---|---|---||hr|0.74|0.728|
|hr|0.74|0.728|
|hr|0.745|0.731|
|hr|0.746|0.733|
|hr|0.739|0.726|
|hr|0.743|0.73|
|hr|0.742|0.727|
|hr|0.744|0.728|
|hr|0.74|0.725|
|hr|0.739|0.724|


Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec



# Model: EMBEDDIA/crosloengual-bert

|language|accuracy|f1 score|
|---|---|---||hr|0.806|0.798|
|hr|0.798|0.789|
|hr|0.798|0.789|
|hr|0.805|0.796|
|hr|0.805|0.796|
|hr|0.805|0.796|
|hr|0.808|0.8|
|hr|0.808|0.798|
|hr|0.809|0.8|
|hr|0.806|0.797|
./finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5 
Accuracies:  [0.7400943396226415, 0.7396226415094339, 0.7452830188679245, 0.7462264150943396, 0.7391509433962264, 0.7433962264150943, 0.7419811320754717, 0.7438679245283019, 0.7396226415094339, 0.7391509433962264] 
F1s:  [0.7276595982785565, 0.7280128370205989, 0.73090755235879, 0.7333423110769459, 0.7262043135735503, 0.7297732989926482, 0.7271033975584644, 0.7283302704921071, 0.7245446739277805, 0.7238516332622682]
EMBEDDIA/crosloengual-bert 
Accuracies:  [0.8061320754716981, 0.7981132075471699, 0.7976415094339623, 0.805188679245283, 0.8047169811320755, 0.8047169811320755, 0.8075471698113208, 0.8080188679245283, 0.809433962264151, 0.8056603773584906] 
F1s:  [0.7979346815022295, 0.7893

## New attepmt:

In [3]:
def train_and_evaluate_HF_model(model_type, model_name, language):
    from simpletransformers.classification import ClassificationModel
    
    train_fname = f"../data/merged-{lang}.train.tsv"
    train = read_file(train_fname, correct_labels = True)
    
    
    model_args = {
        "num_train_epochs": 5,
        "learning_rate": 1e-5,
        "overwrite_output_dir": True,
        "train_batch_size": 40
    }

    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args

    )

    model.overwrite_output_dir = True
    #model.train_model(train, )
    
    test_fname = f"../data/merged-{lang}.test.tsv"
    test = read_file(test_fname, correct_labels = True)
    
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    #y_pred = model.predict(list(test["text"].values))[0]    
    accs = list()
    f1s = list()
    for i in range(5):
        model.train_model(train, )
        y_pred = model.predict(list(test["text"].values))[0]
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro" )
        accs.append(accuracy)
        f1s.append(f1)
    print(f"""
# Model: {model_name}

|language|accuracy|f1 score|
|---|---|---|""", end="")
    for a, f in zip(accs, f1s):
        print(f"|{lang}|{a:0.3}|{f:0.3}|")
    return model, accs, f1s


lang = "hr"
model_type = "bert"
model_name = "finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5_second_attempt"

model, accs, f1s = train_and_evaluate_HF_model(model_type, model_name, lang)



model_name2 = "EMBEDDIA/crosloengual-bert"
model_type2 = "bert"
model2, accs2, f1s2 = train_and_evaluate_HF_model(model_type2, model_name2, lang)
print(model_name, "\nAccuracies: ", accs, "\nF1s: ", f1s)
print(model_name2, "\nAccuracies: ", accs2, "\nF1s: ", f1s2)

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(




# Model: finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5_second_attempt

|language|accuracy|f1 score|
|---|---|---||hr|0.796|0.784|
|hr|0.796|0.785|
|hr|0.793|0.782|
|hr|0.794|0.783|
|hr|0.794|0.783|


Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model chec

/home/peterr/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "




# Model: EMBEDDIA/crosloengual-bert

|language|accuracy|f1 score|
|---|---|---||hr|0.808|0.8|
|hr|0.808|0.799|
|hr|0.805|0.795|
|hr|0.804|0.795|
|hr|0.812|0.804|
finetuned_models/HR_hate___EMBEDDIA/crosloengual-bert_5_second_attempt 
Accuracies:  [0.7957547169811321, 0.7957547169811321, 0.7933962264150943, 0.7938679245283019, 0.7938679245283019] 
F1s:  [0.7837753294802208, 0.7846700440695007, 0.7823304537642092, 0.782680852790697, 0.782680852790697]
EMBEDDIA/crosloengual-bert 
Accuracies:  [0.8080188679245283, 0.8075471698113208, 0.8047169811320755, 0.8042452830188679, 0.8117924528301886] 
F1s:  [0.7999802956806973, 0.7991288523758698, 0.7947127925122843, 0.7949661605607459, 0.8035993778371552]
